<h1> Parameter Tunning with MinMax
    
    GridCV for: learning rate 

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# Import data; set X and y; fill nan values and split in test and training  data:

In [2]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

# Fill nan values 
X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# Apply feature scaling:

In [3]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [4]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Apply Randomized tunning for the whole ANN:

In [5]:
# parameters:
learning_range =  [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
               
for i in learning_range:
    
    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = 15))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
        for layer in range(n_hidden):
            model.add(Dense(n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.1))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = i)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = 7)
    hist_list = pd.DataFrame()
    count = 1
    
    regressor = regressor_tunning()
    
    for train_index, test_index in tscv.split(X_train):
          X_train_split, X_test_split = X_train[train_index], X_train[test_index]
          y_train_split, y_test_split = y_train[train_index], y_train[test_index]
          hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80)
          hist_list = hist_list.append(hist.history, ignore_index = True)
          print(count)
          count = count + 1

    a = []
    b = []
    
    for i in range(len(hist_list.mse)):
        a.append(np.mean(hist_list.mse[i]))
        b.append(np.mean(hist_list.mae[i]))
      
    mse_cv.append(np.mean(a))
    mae_cv.append(np.mean(b))
    
    # predict for X_test  
    y_pred = regressor.predict(X_test)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525
    
    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    
    # smal adjustment
    y_test.replace(0, 0.0001,inplace = True)

    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)



Epoch 1/80
44/44 [==============================] - 0s 3ms/step - loss: 17351.0332 - mse: 17351.0332 - mae: 114.2777
Epoch 2/80
44/44 [==============================] - 0s 3ms/step - loss: 17350.0449 - mse: 17350.0449 - mae: 114.2719
Epoch 3/80
44/44 [==============================] - 0s 2ms/step - loss: 17348.4863 - mse: 17348.4863 - mae: 114.2667
Epoch 4/80
44/44 [==============================] - 0s 2ms/step - loss: 17347.2031 - mse: 17347.2031 - mae: 114.2610
Epoch 5/80
44/44 [==============================] - 0s 2ms/step - loss: 17346.2617 - mse: 17346.2617 - mae: 114.2559
Epoch 6/80
44/44 [==============================] - 0s 2ms/step - loss: 17345.0039 - mse: 17345.0039 - mae: 114.2502
Epoch 7/80
44/44 [==============================] - 0s 2ms/step - loss: 17343.6816 - mse: 17343.6816 - mae: 114.2432
Epoch 8/80
44/44 [==============================] - 0s 2ms/step - loss: 17342.1660 - mse: 17342.1660 - mae: 114.2382
Epoch 9/80
44/44 [==============================] - 0s 3ms/step 

Epoch 70/80
44/44 [==============================] - 0s 1ms/step - loss: 17172.9062 - mse: 17172.9062 - mae: 113.4926
Epoch 71/80
44/44 [==============================] - 0s 1ms/step - loss: 17170.5762 - mse: 17170.5762 - mae: 113.4883
Epoch 72/80
44/44 [==============================] - 0s 1ms/step - loss: 17160.7070 - mse: 17160.7070 - mae: 113.4491
Epoch 73/80
44/44 [==============================] - 0s 2ms/step - loss: 17158.5605 - mse: 17158.5605 - mae: 113.4327
Epoch 74/80
44/44 [==============================] - 0s 3ms/step - loss: 17154.9727 - mse: 17154.9727 - mae: 113.4109
Epoch 75/80
44/44 [==============================] - 0s 3ms/step - loss: 17154.2637 - mse: 17154.2637 - mae: 113.4057
Epoch 76/80
44/44 [==============================] - 0s 3ms/step - loss: 17149.9551 - mse: 17149.9551 - mae: 113.3940
Epoch 77/80
44/44 [==============================] - 0s 3ms/step - loss: 17138.8789 - mse: 17138.8789 - mae: 113.3511
Epoch 78/80
44/44 [==============================] - 0s 

132/132 [==============================] - 0s 1ms/step - loss: 3109.4180 - mse: 3109.4180 - mae: 31.5833
Epoch 50/80
132/132 [==============================] - 0s 1ms/step - loss: 2995.6943 - mse: 2995.6943 - mae: 31.6849
Epoch 51/80
132/132 [==============================] - 0s 1ms/step - loss: 3076.7749 - mse: 3076.7749 - mae: 31.9075
Epoch 52/80
132/132 [==============================] - 0s 2ms/step - loss: 2976.3555 - mse: 2976.3555 - mae: 30.8535
Epoch 53/80
132/132 [==============================] - 0s 2ms/step - loss: 3068.0068 - mse: 3068.0068 - mae: 31.5028
Epoch 54/80
132/132 [==============================] - 0s 3ms/step - loss: 3004.0100 - mse: 3004.0100 - mae: 30.9637
Epoch 55/80
132/132 [==============================] - 0s 3ms/step - loss: 3052.8774 - mse: 3052.8774 - mae: 31.3960
Epoch 56/80
132/132 [==============================] - 0s 3ms/step - loss: 3054.7048 - mse: 3054.7048 - mae: 31.4670
Epoch 57/80
132/132 [==============================] - 0s 2ms/step - loss: 3

220/220 [==============================] - 1s 3ms/step - loss: 3082.0647 - mse: 3082.0649 - mae: 32.0182
Epoch 28/80
220/220 [==============================] - 1s 3ms/step - loss: 3088.8235 - mse: 3088.8235 - mae: 32.0716
Epoch 29/80
220/220 [==============================] - 1s 3ms/step - loss: 3071.6265 - mse: 3071.6265 - mae: 31.4714
Epoch 30/80
220/220 [==============================] - 1s 3ms/step - loss: 3138.6887 - mse: 3138.6887 - mae: 32.2558
Epoch 31/80
220/220 [==============================] - 0s 2ms/step - loss: 3059.1702 - mse: 3059.1702 - mae: 32.1736
Epoch 32/80
220/220 [==============================] - 0s 1ms/step - loss: 3026.8623 - mse: 3026.8623 - mae: 31.7752
Epoch 33/80
220/220 [==============================] - 0s 2ms/step - loss: 3071.7605 - mse: 3071.7605 - mae: 31.8451
Epoch 34/80
220/220 [==============================] - 1s 2ms/step - loss: 3122.1052 - mse: 3122.1050 - mae: 32.1360
Epoch 35/80
220/220 [==============================] - 1s 3ms/step - loss: 3

308/308 [==============================] - 1s 2ms/step - loss: 2947.7549 - mse: 2947.7549 - mae: 32.3317
Epoch 6/80
308/308 [==============================] - 0s 1ms/step - loss: 2962.5991 - mse: 2962.5991 - mae: 32.1440
Epoch 7/80
308/308 [==============================] - 1s 3ms/step - loss: 2956.3315 - mse: 2956.3315 - mae: 32.0964
Epoch 8/80
308/308 [==============================] - 1s 3ms/step - loss: 2904.0571 - mse: 2904.0571 - mae: 31.7530
Epoch 9/80
308/308 [==============================] - 1s 3ms/step - loss: 2958.0842 - mse: 2958.0842 - mae: 32.0843
Epoch 10/80
308/308 [==============================] - 1s 3ms/step - loss: 2938.2190 - mse: 2938.2190 - mae: 31.7293
Epoch 11/80
308/308 [==============================] - 1s 3ms/step - loss: 2985.5054 - mse: 2985.5054 - mae: 32.0886
Epoch 12/80
308/308 [==============================] - 1s 3ms/step - loss: 2886.3159 - mse: 2886.3159 - mae: 31.5239
Epoch 13/80
308/308 [==============================] - 0s 1ms/step - loss: 2924.

...

308/308 [==============================] - 1s 2ms/step - loss: 2310.8037 - mse: 2310.8037 - mae: 26.0065
Epoch 71/80
308/308 [==============================] - 0s 1ms/step - loss: 2231.6238 - mse: 2231.6238 - mae: 25.5204
Epoch 72/80
308/308 [==============================] - 0s 2ms/step - loss: 2306.3804 - mse: 2306.3804 - mae: 25.8060
Epoch 73/80
308/308 [==============================] - 0s 2ms/step - loss: 2246.5510 - mse: 2246.5510 - mae: 25.5358
Epoch 74/80
308/308 [==============================] - 0s 1ms/step - loss: 2261.0691 - mse: 2261.0691 - mae: 25.7810
Epoch 75/80
308/308 [==============================] - 0s 1ms/step - loss: 2336.7400 - mse: 2336.7400 - mae: 25.9475
Epoch 76/80
308/308 [==============================] - 0s 2ms/step - loss: 2229.7742 - mse: 2229.7742 - mae: 25.2914
Epoch 77/80
308/308 [==============================] - 0s 1ms/step - loss: 2303.5847 - mse: 2303.5847 - mae: 26.1637
Epoch 78/80
308/308 [==============================] - 0s 1ms/step - loss: 2

88/88 [==============================] - 0s 1ms/step - loss: 223374416.0000 - mse: 223374416.0000 - mae: 7923.6079
Epoch 37/80
88/88 [==============================] - 0s 1ms/step - loss: 96622400.0000 - mse: 96622400.0000 - mae: 6166.6733
Epoch 38/80
88/88 [==============================] - 0s 1ms/step - loss: 83623976.0000 - mse: 83623976.0000 - mae: 5402.8462
Epoch 39/80
88/88 [==============================] - 0s 1ms/step - loss: 55018564.0000 - mse: 55018564.0000 - mae: 4749.5415
Epoch 40/80
88/88 [==============================] - 0s 1ms/step - loss: 47678940.0000 - mse: 47678940.0000 - mae: 4428.5469
Epoch 41/80
88/88 [==============================] - 0s 1ms/step - loss: 39135796.0000 - mse: 39135796.0000 - mae: 3996.7300
Epoch 42/80
88/88 [==============================] - 0s 1ms/step - loss: 62720244.0000 - mse: 62720244.0000 - mae: 4908.4341
Epoch 43/80
88/88 [==============================] - 0s 1ms/step - loss: 1718068480.0000 - mse: 1718068480.0000 - mae: 20104.7148
Epoch

176/176 [==============================] - 0s 3ms/step - loss: 4306.1948 - mse: 4306.1948 - mae: 43.8543
Epoch 11/80
176/176 [==============================] - 0s 1ms/step - loss: 3902.5864 - mse: 3902.5864 - mae: 41.5631
Epoch 12/80
176/176 [==============================] - 0s 1ms/step - loss: 4139.7778 - mse: 4139.7778 - mae: 44.4433
Epoch 13/80
176/176 [==============================] - 0s 2ms/step - loss: 3915.3938 - mse: 3915.3938 - mae: 41.9983
Epoch 14/80
176/176 [==============================] - 0s 3ms/step - loss: 5862.6709 - mse: 5862.6709 - mae: 53.4956
Epoch 15/80
176/176 [==============================] - 0s 3ms/step - loss: 25146.9297 - mse: 25146.9297 - mae: 91.1098
Epoch 16/80
176/176 [==============================] - 0s 2ms/step - loss: 3794.9021 - mse: 3794.9021 - mae: 40.6630
Epoch 17/80
176/176 [==============================] - 0s 3ms/step - loss: 3991.1807 - mse: 3991.1807 - mae: 42.0175
Epoch 18/80
176/176 [==============================] - 0s 3ms/step - loss:

220/220 [==============================] - 0s 1ms/step - loss: 130209.7422 - mse: 130209.7422 - mae: 249.8639
Epoch 64/80
220/220 [==============================] - 0s 2ms/step - loss: 34119.2969 - mse: 34119.2969 - mae: 134.5993
Epoch 65/80
220/220 [==============================] - 1s 2ms/step - loss: 28148.3359 - mse: 28148.3359 - mae: 117.0533
Epoch 66/80
220/220 [==============================] - 1s 3ms/step - loss: 13953.7021 - mse: 13953.7021 - mae: 86.1120
Epoch 67/80
220/220 [==============================] - 1s 3ms/step - loss: 13211.2422 - mse: 13211.2422 - mae: 83.3478
Epoch 68/80
220/220 [==============================] - 1s 3ms/step - loss: 133797.6094 - mse: 133797.6094 - mae: 232.3799
Epoch 69/80
220/220 [==============================] - 1s 3ms/step - loss: 30024.9648 - mse: 30024.9648 - mae: 107.2535
Epoch 70/80
220/220 [==============================] - 1s 3ms/step - loss: 10399.2451 - mse: 10399.2451 - mae: 72.8153
Epoch 71/80
220/220 [==============================

308/308 [==============================] - 1s 3ms/step - loss: 957204160.0000 - mse: 957204160.0000 - mae: 17667.0840
Epoch 34/80
308/308 [==============================] - 1s 2ms/step - loss: 203790848.0000 - mse: 203790848.0000 - mae: 10144.0098
Epoch 35/80
308/308 [==============================] - 0s 1ms/step - loss: 56710436.0000 - mse: 56710436.0000 - mae: 5702.8784
Epoch 36/80
308/308 [==============================] - 1s 3ms/step - loss: 128373968.0000 - mse: 128373968.0000 - mae: 5952.0444
Epoch 37/80
308/308 [==============================] - 1s 3ms/step - loss: 332642496.0000 - mse: 332642496.0000 - mae: 11285.6152
Epoch 38/80
308/308 [==============================] - 1s 3ms/step - loss: 33737408.0000 - mse: 33737408.0000 - mae: 4304.1987
Epoch 39/80
308/308 [==============================] - 1s 3ms/step - loss: 16914666.0000 - mse: 16914666.0000 - mae: 2907.0706
Epoch 40/80
308/308 [==============================] - 1s 3ms/step - loss: 7885999.5000 - mse: 7885999.5000 - ma

In [6]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = learning_range)

results

,rmse_cv,mae_cv,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0.00001,83.679212,56.674975,30.446977,23.158006,63.078432,52.723361,20.748108,18.347769
0.00010,59.658602,33.258136,30.002768,23.174921,61.736288,51.010878,20.652311,18.646055
0.00100,54.213840,28.327891,31.641372,24.781135,60.495429,49.358767,23.846160,20.782394
0.01000,50.778365,25.764503,31.853014,25.086374,61.246237,50.629960,23.861944,20.930473
0.10000,52.445007,26.697781,31.535514,25.843463,60.553469,50.578956,23.658411,21.819038
1.00000,473161.222523,21473.738521,2265.421614,2250.734681,2294.970220,2281.509458,2260.577571,2245.727674


In [8]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_cv,mae_cv,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
1e-05,83.679212,56.674975,30.446977,23.158006,63.078432,52.723361,20.748108,18.347769
0.0001,59.658602,33.258136,30.002768,23.174921,61.736288,51.010878,20.652311,18.646055
0.001,54.213840,28.327891,31.641372,24.781135,60.495429,49.358767,23.846160,20.782394
0.01,50.778365,25.764503,31.853014,25.086374,61.246237,50.629960,23.861944,20.930473
0.1,52.445007,26.697781,31.535514,25.843463,60.553469,50.578956,23.658411,21.819038
1.0,473161.222523,21473.738521,2265.421614,2250.734681,2294.970220,2281.509458,2260.577571,2245.727674


In [9]:
results.to_csv('Results_tuning_7_best_learning_rate_with_spike_normal_regions.csv')